In [2]:
from pydantic import BaseModel
from langchain.text_splitter import TokenTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceEmbeddings
import ollama
from langchain.llms.base import LLM

e:\Finalyear_project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# load the pdf



In [4]:
loader = PyPDFLoader("example1.pdf")
documents = loader.load()


# Embeddings 

In [6]:

token_splitter = TokenTextSplitter(chunk_size=500, chunk_overlap=50) 
embedding_model = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')
texts = token_splitter.split_documents(documents)

C:\Users\rajpa\AppData\Local\Temp\ipykernel_10156\298322004.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')


In [7]:
db = FAISS.from_documents(texts,embedding = embedding_model)

In [8]:
# Define a custom LLM wrapper for Ollama to integrate with LangChain
class OllamaLLM(LLM, BaseModel):
    model_name: str  # Explicitly declare the model_name field

    def _call(self, prompt: str, stop=None):
        response = ollama.generate(model=self.model_name, prompt=prompt)
        print(response)
        return response["response"]

    @property
    def _llm_type(self):
        return "ollama"

In [17]:

# Define the prompt template
prompt_template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Answer:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

# Initialize the Ollama model using the custom LLM wrapper
ollama_llm = OllamaLLM(model_name="llama3.2")

# Define the RetrievalQA chain
qa = RetrievalQA.from_chain_type(
    llm=ollama_llm,
    chain_type="stuff",
    retriever=db.as_retriever(),
    chain_type_kwargs={"prompt": PROMPT},
    return_source_documents=True
)

# Define the RAG pipeline function
def rag_pipeline(query):
    result = qa({"query": query})
    answer = result['result']
    source_documents = result['source_documents']
    context = [doc.page_content for doc in source_documents]
    return  answer

In [18]:
print(rag_pipeline("Write a R program to find the sum of all even numbers between two given numbers using for loop."))

model='llama3.2' created_at='2024-12-15T15:41:17.8450383Z' done=True done_reason='stop' total_duration=41647788800 load_duration=38839800 prompt_eval_count=1175 prompt_eval_duration=452975000 eval_count=290 eval_duration=41152422000 response='Here is a simple R program that uses a `for` loop to calculate the sum of all even numbers between two given numbers:\n\n```r\n# Function to print Fibonacci series\nfibonacci_series <- function(n) {\n  fib <- numeric(n)\n  fib[1] <- 0\n  fib[2] <- 1\n  for (i in 3:n) {\n    fib[i] <- fib[i-1] + fib[i-2]\n  }\n  return(fib)\n}\n\n# Example usage:\nn_terms <- 10\nfibonacci_result <- fibonacci_series(n_terms)\n\n# Function to calculate sum of even numbers between two given numbers\nsum_even_numbers <- function(a, b) {\n  total_sum <- 0\n  for (i in a:b) {\n    if (i %% 2 == 0) {\n      total_sum <- total_sum + i\n    }\n  }\n  return(total_sum)\n}\n\n# Call the functions with given values\na <- 1\nb <- 20\nsum_result <- sum_even_numbers(a, b)\n\n# Pr